# How to combine colormaps for SatPy?

## 1. `proplot` method (recommended for interactive use)

Let's read the file first. You can check the basic tutorial [here](https://github.com/zxdawn/FY-4/blob/master/satpy/examples/FY4A_agri_introduction%28EN%29.ipynb).

In [1]:
%matplotlib inline
import os, glob
from satpy.scene import Scene
from trollimage.colormap import greys, spectral
import warnings
warnings.simplefilter('ignore')

# set the config path
os.environ['PPP_CONFIG_DIR'] = '../satpy_config/'

# load FY4A filenames
filenames = glob.glob('../data/FY4A-_AGRI*4000M_V0001.HDF')

# create the scene object
scn = Scene(filenames, reader='agri_l1')
channel = 'C12'
scn.load([channel])

# resample it to interested region
lekima_scene = scn.resample('lekima_4km')
img = lekima_scene[channel]

# get lon.lat from scene
lon, lat = img.attrs['area'].get_lonlats()

Let's try `cartopy` first:

In [2]:
%%time
# %%capture

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# get projection and set axis
crs = img.attrs['area'].to_cartopy_crs()
f, ax = plt.subplots(subplot_kw=dict(projection=crs))
ax.gridlines()

# plot data
plt.imshow(img, transform=crs, extent=crs.bounds, origin='upper')

# set others
cbar = plt.colorbar()
cbar.set_label(r'Brightness Temperature ($^\circ$C)')
ax.set_title(img.attrs['long_name'])

# f.savefig('./figures/pyplot_crs.jpg')

Wall time: 1.13 s


<img src='./figures/pyplot_crs.jpg'>

Of course, we can switch to `pcarree` projection:

In [3]:
%%time
# %%capture

# set axis
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.gridlines()

# plot data
plt.pcolormesh(lon, lat, img, transform=ccrs.PlateCarree())

# set others
cbar = plt.colorbar()
cbar.set_label(r'Brightness Temperature ($^\circ$C)')
ax.set_title(img.attrs['long_name'])

# f.savefig('./figures/pyplot_pcarree.jpg')

Wall time: 1.25 s


<img src='./figures/pyplot_pcarree.jpg'>

How about `proplot`? (The combined colormap is shown too!)

In [4]:
# create our own colormap first
import proplot as plot

ir_br = plot.Colormap('spectral_r', 'grays',
                      ratios=(1, 1), name='ir_br',
#                       save=True,
                      )

In [5]:
%%time
# %%capture

# set axis
f, axs = plot.subplots(projection=crs)
axs.format(geogrid=False)

# plot data
m = axs.pcolormesh(img-273.15, transform=crs, cmap=ir_br, levels=256)

# set others
axs.colorbar(m, loc='r', label='Brightness Temperature ($^\circ$C)')
axs.set_title(img.attrs['long_name'])

# f.savefig('./figures/proplot_crs.jpg')

Wall time: 3.47 s


<img src='./figures/proplot_crs.jpg'>

If you prefer 'pure' (no border/frame) image, it's also possible:

In [6]:
%%time
# %%capture

# set axis
f, axs = plot.subplots(projection=crs)
axs.format(geogrid=False, linewidth=0)
f.patch.set_visible(False)

# plot data
m = axs.pcolormesh(img-273.15, transform=crs, cmap=ir_br, levels=256)

# set others
axs.format(title='')

# f.savefig('./figures/proplot_crs_noborder.jpg')

Wall time: 2.95 s


<img src='./figures/proplot_crs_noborder.jpg'>

More scientific figure:

In [7]:
%%time
# %%capture

# set axis
f, axs = plot.subplots(proj='pcarree')
axs.format(labels=True,
           lonlines=10,
           latlines=5,
           lonlim=(lon.min(), lon.max()),
           latlim=(lat.min(), lat.max()),
           geogridlinewidth=0.5
           )

# plot data
m = axs.pcolormesh(lon, lat, img-273.15, transform=ccrs.PlateCarree(), cmap=ir_br, levels=256)

# set others
axs.format(title=img.attrs['long_name'])
cb = axs.colorbar(m, loc='r', label='Brightness Temperature ($^\circ$C)')

# f.savefig('./figures/proplot_pcarree.jpg')

Wall time: 3.29 s


<img src='./figures/proplot_pcarree.jpg'>

Of course, we can set the limit of colorbar by ourself:

In [9]:
%%time
# %%capture

# set axis
f, axs = plot.subplots(proj='pcarree')
axs.format(labels=True,
           lonlines=10,
           latlines=5,
           lonlim=(lon.min(), lon.max()),
           latlim=(lat.min(), lat.max()),
           geogridlinewidth=0.5
           )


# set levels
cb_levels = plot.arange(-90, 30, 10)

# plot data
m = axs.pcolormesh(lon, lat, img-273.15,
                   cmap=ir_br,
                   vmin=-90, vmax=30,
                   levels=256)

# set others
axs.format(title=img.attrs['long_name'])
axs.colorbar(m, loc='r', values=cb_levels, label='Brightness Temperature ($^\circ$C)')

# f.savefig('./figures/proplot_pcarree_levels.jpg')

Wall time: 2.87 s


<img src='./figures/proplot_pcarree_levels.jpg'>

## 2. `enhancement` method (recommended for batch production)

Add the configuration to `$PPP_CONFIG_DIR/enhancements/<sensor>.yaml` first.

I will use `agri` as an example below. For more information see [composites](https://satpy.readthedocs.io/en/latest/composites.html) and [enhancements](https://satpy.readthedocs.io/en/latest/enhancements.html) sections of official Satpy Doc.

Here's the content of `agri.yaml`:

```
enhancements:
  AGRI_C12:
    name: C12
    operations:
      - name: colorize
        method: &colorizefun !!python/name:satpy.enhancements.colorize
        kwargs:
          palettes:
            - {colors: spectral, min_value: 183.15, max_value: 253.15}
            - {colors: greys, min_value: 253.15, max_value: 303.15}
```

Some explanations can be found in this [issue](https://github.com/pytroll/satpy/issues/459) of satpy repository.

We will update the doc for this soon ...

I just mention some important things here:

1. The filename should be <sensor>.yaml in case you mess up something;
2. `standard_name` should be set correctly;
    > Note that if you removed the first `name:` part then the enhancement would be used for all datasets matching standard_name.
    > 
    > If you remove `standard_name` it would match for only datasets matching the exact name.
    > 
    > If you have both it must match both of those items.
    > 
    > You could accomplish the same thing in generic.yaml by specifying sensor: abi in addition to name and/or standard_name.

Now, let's read the data and apply our own colormap to it:

Let make the figure more beautiful by adding colorbar:

In [10]:
# set the corresponding colormap ticks range
greys.set_range(-40, 30)
spectral.set_range(-90, -40)

# set the right width and height for colorbar
height, width = lekima_scene[channel].shape
h_legend = int(height/20)
w_legend = int(width/2)

# save dataset to image
lekima_scene.save_datasets(base_dir='./figures/',
                           filename='enhancement.jpg',
                           compute=True,
                           fill_value=0,
                           datasets=[channel],
                           writer='simple_image',
                           decorate={'decorate': [
                                    {'scale': {'colormap': spectral, 'extend': False,
                                               'width': w_legend, 'height': h_legend,
                                               'tick_marks': 5, 'minor_tick_marks': 1,
                                               'cursor': [0, 0], 'bg':'white',
                                                # title settings
                                                'title':'Brightness Temperature',
                                                'fontsize': h_legend*10, 'align': [w_legend, 0],
                                    }},
                                    {'scale': {'colormap': greys, 'extend': False,
                                               'width': w_legend, 'height': h_legend,
                                               'cursor': [w_legend, 0],
                                    }},
                        ]
                     }
                 )

<img src='./figures/enhancement.jpg'>